In [47]:
# GRUPO
# Alexandre Galocha Pinto Junior  10734706
# Eduardo Pirro                   10734665
# Fernando Gorgulho Fayet         10734407

In [48]:
import glfw
from OpenGL.GL import *
import OpenGL.GL.shaders
import numpy as np
import glm
import math
from PIL import Image
from lib import Object3D
from lib import Transform3D

### Código base usado em aula

In [49]:
glfw.init()
glfw.window_hint(glfw.VISIBLE, glfw.FALSE);
altura = 1080
largura = 1920
window = glfw.create_window(largura, altura, "T3 - CG", None, None)
glfw.make_context_current(window)


vertex_code = """
        attribute vec3 position;
        attribute vec2 texture_coord;
        attribute vec3 normals;
        
       
        varying vec2 out_texture;
        varying vec3 out_fragPos;
        varying vec3 out_normal;
                
        uniform mat4 model;
        uniform mat4 view;
        uniform mat4 projection;
        uniform mat4 norms_model;
        
        void main(){
            gl_Position = projection * view * model * vec4(position,1.0);
            out_texture = vec2(texture_coord);
            out_fragPos = vec3(model * vec4(position, 1.0));
            out_normal = vec3(norms_model * vec4(normals, 1.0));           
        }
        """

fragment_code = """

        // parametros da iluminacao ambiente e difusa
        uniform vec3 lightPos1; // define coordenadas de posicao da luz #1
        uniform vec3 lightPos2; // define coordenadas de posicao da luz #2
        uniform mat4 lightPos2Model; // define transformacoes na posicao da luz #2
        uniform float ka; // coeficiente de reflexao ambiente
        uniform float kd; // coeficiente de reflexao difusa
        
        // parametros da iluminacao especular
        uniform vec3 viewPos; // define coordenadas com a posicao da camera/observador
        uniform float ks; // coeficiente de reflexao especular
        uniform float ns; // expoente de reflexao especular
        
        // parametro com a cor da(s) fonte(s) de iluminacao
        uniform float ambientIntensity;
        uniform float sunIntensity;
        vec3 lightColor = vec3(1, 1, 1);
        uniform float light1Intensity;
        uniform float light2Intensity;
        uniform float light1Enabled;
        uniform float light2Enabled;

        // parametros recebidos do vertex shader
        varying vec2 out_texture; // recebido do vertex shader
        varying vec3 out_normal; // recebido do vertex shader
        varying vec3 out_fragPos; // recebido do vertex shader
        uniform sampler2D samplerTexture;
        
        void main(){
        
            // calculando reflexao ambiente
            vec3 ambient =  ka * ambientIntensity * vec3(1, 1, 1);
        
            ////////////////////////
            // Luz #1
            ////////////////////////
            
            // calculando reflexao difusa
            vec3 norm1 = normalize(out_normal); // normaliza vetores perpendiculares
            vec3 lightDir1 = normalize(lightPos1 - out_fragPos); // direcao da luz
            float diff1 = max(dot(norm1, lightDir1), 0.0); // verifica limite angular (entre 0 e 90)
            vec3 diffuse1 = kd * diff1 * lightColor; // iluminacao difusa
            
            // calculando reflexao especular
            vec3 viewDir1 = normalize(viewPos - out_fragPos); // direcao do observador/camera
            vec3 reflectDir1 = reflect(-lightDir1, norm1); // direcao da reflexao
            float spec1 = pow(max(dot(viewDir1, reflectDir1), 0.0), ns);
            vec3 specular1 = ks * spec1 * lightColor;
            
            
            ////////////////////////
            // Luz #2
            ////////////////////////

            // aplicando transformacao na luz #2
            vec3 newLightPos = vec3(lightPos2Model * vec4(lightPos2, 1.0));
            
            // this is line 58
            vec3 norm2 = normalize(out_normal); // normaliza vetores perpendiculares
            vec3 lightDir2 = normalize(newLightPos - out_fragPos); // direcao da luz
            
            // calculando reflexao difusa
            float diff2 = max(dot(norm2, lightDir2), 0.0); // verifica limite angular (entre 0 e 90)
            vec3 diffuse2 = kd * diff2 * lightColor * sunIntensity; // iluminacao difusa
            
            // calculando reflexao especular
            vec3 viewDir2 = normalize(viewPos - out_fragPos); // direcao do observador/camera
            vec3 reflectDir2 = reflect(-lightDir2, norm2); // direcao da reflexao
            float spec2 = pow(max(dot(viewDir2, reflectDir2), 0.0), ns);
            vec3 specular2 = ks * spec2 * lightColor * sunIntensity;    
            
            ////////////////////////
            // Combinando as duas fontes
            ////////////////////////
            
            // aplicando o modelo de iluminacao
            vec4 texture = texture2D(samplerTexture, out_texture);
            vec3 light1Sum = light1Enabled * light1Intensity * (diffuse1 + specular1);
            vec3 light2Sum = light2Enabled * light2Intensity * (diffuse2 + specular2);
            vec4 result = vec4((ambient + light1Sum + light2Sum),1.0) * texture; // aplica iluminacao
            gl_FragColor = result;

        }
        """

# Request a program and shader slots from GPU
program  = glCreateProgram()
vertex   = glCreateShader(GL_VERTEX_SHADER)
fragment = glCreateShader(GL_FRAGMENT_SHADER)

# Set shaders source
glShaderSource(vertex, vertex_code)
glShaderSource(fragment, fragment_code)

# Compile shaders
glCompileShader(vertex)
if not glGetShaderiv(vertex, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(vertex).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Vertex Shader")
    
glCompileShader(fragment)
if not glGetShaderiv(fragment, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(fragment).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Fragment Shader")
    
# Attach shader objects to the program
glAttachShader(program, vertex)
glAttachShader(program, fragment)

# Build program
glLinkProgram(program)
if not glGetProgramiv(program, GL_LINK_STATUS):
    print(glGetProgramInfoLog(program))
    raise RuntimeError('Linking error')
    
# Make program the default program
glUseProgram(program)

### Função de carregamento de arquivos de textura

In [50]:
# glHint(GL_LINE_SMOOTH_HINT, GL_DONT_CARE)
# glEnable( GL_BLEND )
# glBlendFunc( GL_SRC_ALPHA, GL_ONE_MINUS_SRC_ALPHA )
# glEnable(GL_LINE_SMOOTH)
glEnable(GL_TEXTURE_2D)
qtd_texturas = 15
textures = glGenTextures(qtd_texturas)

def load_texture_from_file(texture_id, img_textura):
    glBindTexture(GL_TEXTURE_2D, texture_id)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_WRAP_S, GL_REPEAT)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_WRAP_T, GL_REPEAT)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MIN_FILTER, GL_LINEAR)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MAG_FILTER, GL_LINEAR)
    img = Image.open(img_textura)
    img_width = img.size[0]
    img_height = img.size[1]
    image_data = img.tobytes("raw", "RGB", 0, -1)
#     image_data = img.convert("RGBA").tobytes("raw", "RGBA",0,-1)

#     glTexImage2D(GL_TEXTURE_2D, 0, GL_RGBA, img_width, img_height, 0, GL_RGBA, GL_UNSIGNED_BYTE, image_data)
    glTexImage2D(GL_TEXTURE_2D, 0, GL_RGB, img_width, img_height, 0, GL_RGB, GL_UNSIGNED_BYTE, image_data)

### Criação dos modelos

As texturas referentes a cada modelo devem ser carregadas no código principal com a função 'load_texture_from_file'

Para cada modelo, cria-se um object3d e carrega-se os vértices através da função 'load_from_file'

Para cada modelo, deve-se usar a função 'bind_textures' para indicar o id das texturas usadas pelo modelo

In [51]:
### Chão de pedra
load_texture_from_file(0, 'models/stone_floor/txt_stone_floor.jpg')

stone_floor = Object3D.Object3D()
stone_floor.t_y = -1.01
stone_floor.s_x = 10.0
stone_floor.s_y = 10.0
stone_floor.s_z = 10.0

# sofre influencia apenas da luz externa
stone_floor.light_one_enabled = False
stone_floor.light_two_enabled = True

stone_floor.load_from_file('models/stone_floor/mdl_stone_floor.obj')
stone_floor.bind_textures([0])

In [52]:
### Casa
load_texture_from_file(3, 'models/house/txt_house_1.jpg')
load_texture_from_file(2, 'models/house/txt_house_2.png')
load_texture_from_file(1, 'models/house/txt_house_3.png')

house = Object3D.Object3D()
house.t_y = -1.0
house.s_x = 0.02
house.s_y = 0.02
house.s_z = 0.02
# house.ks = 1
# house.ns = 0.00000001

# sofre influencia apenas da luz externa
house.light_one_enabled = False
house.light_two_enabled = True

house.load_from_file('models/house/mdl_house.obj')
house.bind_textures([1,2,3])

In [53]:
### Casa
inner_house = Object3D.Object3D()
inner_house.t_y = -1.0
inner_house.s_x = 0.0199
inner_house.s_y = 0.0199
inner_house.s_z = 0.019
inner_house.invert_norms = True
# house.ks = 1
# house.ns = 0.00000001

# sofre influencia apenas da luz interna
inner_house.light_one_enabled = True
inner_house.light_two_enabled = False

inner_house.load_from_file('models/house/mdl_house.obj')
inner_house.bind_textures([1,2,3])

In [54]:
### Mesa (dentro da casa)
load_texture_from_file(4, 'models/table/txt_table.png')

table = Object3D.Object3D()
table.t_y = -1.0

# sofre influencia apenas da luz interna
table.light_one_enabled = True
table.light_two_enabled = False

table.load_from_file('models/table/mdl_table.obj')
table.bind_textures([4,4])

In [55]:
### Homem Aranha
load_texture_from_file(5, 'models/spiderman/txt_spiderman.png')

spiderman = Object3D.Object3D()
spiderman.t_y = -1.0
spiderman.t_z = -1.1
spiderman.t_x = -1.3
spiderman.a_y = 55
spiderman.s_x = 0.15
spiderman.s_y = 0.15
spiderman.s_z = 0.15

# sofre influencia apenas da luz interna
spiderman.light_one_enabled = True
spiderman.light_two_enabled = False

spiderman.load_from_file('models/spiderman/mdl_spiderman.obj')
spiderman.bind_textures([5])

In [56]:
### Árvore
load_texture_from_file(6, 'models/tree/txt_tree_1.jpg')
load_texture_from_file(7, 'models/tree/txt_tree_2.png')

tree = Object3D.Object3D()
tree.t_y = -1.0
tree.t_z = -3.1
tree.t_x = +7.0
tree.s_x = 0.75
tree.s_y = 0.75
tree.s_z = 0.75

# sofre influencia apenas da luz externa
tree.light_one_enabled = False
tree.light_two_enabled = True

tree.load_from_file('models/tree/mdl_tree.obj')
tree.bind_textures([6,7])

In [57]:
### Tanques
load_texture_from_file(8, 'models/tanks/txt_tanks.jpg')

tanks = Object3D.Object3D()
tanks.t_y = -1.0
tanks.t_z = -7
tanks.t_x = -7.0
tanks.a_x = -90
tanks.s_x = 0.5
tanks.s_y = 0.5
tanks.s_z = 0.5

# sofre influencia apenas da luz externa
tanks.light_one_enabled = False
tanks.light_two_enabled = True


tanks.load_from_file('models/tanks/mdl_tanks.obj')
tanks.bind_textures([8,8,8,8,8])

In [58]:
### Avião
load_texture_from_file(9, 'models/plane/txt_plane_1.jpg')
load_texture_from_file(10, 'models/plane/txt_plane_2.jpg')

plane = Object3D.Object3D()
plane.t_y = 10.0
plane.t_x = -20.0
# plane.t_y = 30
# plane.t_x = 0
plane.a_x = -90
plane.a_y = +30
plane.s_x = 0.0005
plane.s_y = 0.0005
plane.s_z = 0.0005
plane.ks = 0.5
plane.ns = 0.00000001

# sofre influencia apenas da luz externa
plane.light_one_enabled = False
plane.light_two_enabled = True

plane.load_from_file('models/plane/mdl_plane.obj')
plane.bind_textures([9,10])

In [59]:
### Avião no chão
ground_plane = Object3D.Object3D()
ground_plane.t_y = -0.2
ground_plane.t_x = -20.0
# plane.t_y = 30
# plane.t_x = 0
ground_plane.a_x = -90
ground_plane.s_x = 0.007
ground_plane.s_y = 0.007
ground_plane.s_z = 0.007
ground_plane.ks = 0.5
ground_plane.ns = 0.00000001

# sofre influencia apenas da luz externa
ground_plane.light_one_enabled = False
ground_plane.light_two_enabled = True

ground_plane.load_from_file('models/plane/mdl_plane.obj')
ground_plane.bind_textures([9,10])

In [60]:
### Sky box
load_texture_from_file(11, 'models/sky/txt_sky.png')

sky_box = Object3D.Object3D()
sky_box.s_x = 2.0
sky_box.s_y = 2.0
sky_box.s_z = 2.0
sky_box.invert_norms = True

# sofre influencia apenas da luz externa
sky_box.light_one_enabled = False
sky_box.light_two_enabled = True

sky_box.load_from_file('models/sky/mdl_sky.obj')
sky_box.bind_textures([11])

In [61]:
### Notebook
load_texture_from_file(13, 'models/notebook/txt_notebook_1.jpg')
load_texture_from_file(12, 'models/notebook/txt_notebook_2.jpg')

notebook = Object3D.Object3D()
notebook.t_y = -0.64
notebook.s_x = 0.1
notebook.s_y = 0.1
notebook.s_z = 0.1

# sofre influencia apenas da luz interna
notebook.light_one_enabled = True
notebook.light_two_enabled = False

notebook.load_from_file('models/notebook/mdl_notebook.obj')
notebook.bind_textures([12, 13])

In [62]:
### Sun

sun = Object3D.Object3D()
sun.s_x = 0.020
sun.s_y = 0.020
sun.s_z = 0.020
sun.t_y = 30
sun.t_x = 0
sun.ka = 10

sun.light_one_enabled = True
sun.light_two_enabled = True

sun.load_from_file('models/sky/mdl_sky.obj')
sun.bind_textures([11])

In [63]:
### Lamp

lamp = Object3D.Object3D()
lamp.s_x = 0.0002
lamp.s_y = 0.0002
lamp.s_z = 0.0002
lamp.t_x = -0.1
lamp.t_y = -0.5
lamp.ka = 10

lamp.light_one_enabled = True
lamp.light_two_enabled = True

lamp.load_from_file('models/sky/mdl_sky.obj')
lamp.bind_textures([11])

In [64]:
### Chão de grama
load_texture_from_file(14, 'models/grass/txt_grass.jpg')

grass = Object3D.Object3D()
grass.t_y = -1.011
grass.s_x = 100.0
grass.s_y = 100.0
grass.s_z = 100.0

# sofre influencia apenas da luz externa
grass.light_one_enabled = False
grass.light_two_enabled = True

grass.load_from_file('models/grass/mdl_grass.obj')
grass.bind_textures([14])

In [65]:
all_objects = [stone_floor, grass, sky_box, house, inner_house, table, spiderman, tree, tanks, plane, ground_plane, notebook, sun, lamp]
# all_objects = [stone_floor, sky_box, house, table, spiderman, tree, tanks, plane, notebook, notebook2]

### Preparando os dados e enviando para a GPU

In [66]:
# Request a buffer slot from GPU
buffer = glGenBuffers(3)

serialized_vertices, serialized_tex_coords, serialized_normals = Object3D.serialize_objects3d(all_objects)



vertices = np.zeros(len(serialized_vertices), [("position", np.float32, 3)])
vertices['position'] = serialized_vertices

# Upload data
glBindBuffer(GL_ARRAY_BUFFER, buffer[0])
glBufferData(GL_ARRAY_BUFFER, vertices.nbytes, vertices, GL_STATIC_DRAW)
stride = vertices.strides[0]
offset = ctypes.c_void_p(0)
loc_vertices = glGetAttribLocation(program, "position")
glEnableVertexAttribArray(loc_vertices)
glVertexAttribPointer(loc_vertices, 3, GL_FLOAT, False, stride, offset)



textures = np.zeros(len(serialized_tex_coords), [("position", np.float32, 2)]) # duas coordenadas
textures['position'] = serialized_tex_coords

# Upload data
glBindBuffer(GL_ARRAY_BUFFER, buffer[1])
glBufferData(GL_ARRAY_BUFFER, textures.nbytes, textures, GL_STATIC_DRAW)
stride = textures.strides[0]
offset = ctypes.c_void_p(0)
loc_texture_coord = glGetAttribLocation(program, "texture_coord")
glEnableVertexAttribArray(loc_texture_coord)
glVertexAttribPointer(loc_texture_coord, 2, GL_FLOAT, False, stride, offset)



normals = np.zeros(len(serialized_normals), [("position", np.float32, 3)]) # três coordenadas
normals['position'] = serialized_normals

# Upload coordenadas normals de cada vertice
glBindBuffer(GL_ARRAY_BUFFER, buffer[2])
glBufferData(GL_ARRAY_BUFFER, normals.nbytes, normals, GL_STATIC_DRAW)
stride = normals.strides[0]
offset = ctypes.c_void_p(0)
loc_normals_coord = glGetAttribLocation(program, "normals")
glEnableVertexAttribArray(loc_normals_coord)
glVertexAttribPointer(loc_normals_coord, 3, GL_FLOAT, False, stride, offset)

### Eventos para modificar a posição da câmera.

* Movimentação com as teclas A, S, D e W
* Foco da câmera com o mouse
* A janela captura o mouse! Para fecha-la, precione Alt-F4

In [67]:
cameraPos   = glm.vec3(-1.0,  -0.2,  0.7);
cameraFront = glm.vec3(0.0,  0.0, -1.0);
cameraUp    = glm.vec3(0.0,  1.0,  0.0);

skybox_radius = 100                    # raio a partir da origem em que o jogador pode se movimentar

polygonal_mode = False
ambient_intensity = 1
light_one_intensity = 1
light_two_intensity = 1

sun_angle = 0

def key_event(window,key,scancode,action,mods):
    global cameraPos, cameraFront, cameraUp, polygonal_mode, ambient_intensity, sun_angle, light_one_intensity, light_two_intensity
    
    cameraSpeed = 0.2
    if key == 87 and (action==1 or action==2): # tecla W
        newCamPos = cameraPos + cameraSpeed * cameraFront
        if(glm.length(newCamPos) < skybox_radius and newCamPos[1] > -0.9):
            cameraPos = newCamPos
    
    if key == 83 and (action==1 or action==2): # tecla S
        newCamPos = cameraPos - cameraSpeed * cameraFront
        if(glm.length(newCamPos) < skybox_radius and newCamPos[1] > -0.9):
            cameraPos = newCamPos
    
    if key == 65 and (action==1 or action==2): # tecla A
        newCamPos = cameraPos - glm.normalize(glm.cross(cameraFront, cameraUp)) * cameraSpeed
        if(glm.length(newCamPos) < skybox_radius and newCamPos[1] > -0.9):
            cameraPos = newCamPos
        
    if key == 68 and (action==1 or action==2): # tecla D
        newCamPos = cameraPos + glm.normalize(glm.cross(cameraFront, cameraUp)) * cameraSpeed
        if(glm.length(newCamPos) < skybox_radius and newCamPos[1] > -0.9):
            cameraPos = newCamPos
    
    if key == 85 and action==1: # tecla U
        ambient_intensity += 1

    if key == 80 and action==1: # tecla P
        # polygonal_mode = not polygonal_mode
        ambient_intensity = max(ambient_intensity - 1, 0)

    if key == 49 and action==1: # tecla 1
        light_one_intensity += (-1 if mods == 1 else 1)
        light_one_intensity = max(light_one_intensity, 0)

    if key == 50 and action==1: # tecla 2
        light_two_intensity += (-1 if mods == 1 else 1)
        light_two_intensity = max(light_two_intensity, 0)

    if key == 262: sun_angle -= 12.25
    if key == 263: sun_angle += 12.25
            
        
        
        
firstMouse = True
yaw = -90.0 
pitch = 0.0
lastX =  largura/2
lastY =  altura/2

def mouse_event(window, xpos, ypos):
    global firstMouse, cameraFront, yaw, pitch, lastX, lastY, fov
    if firstMouse:
        lastX = xpos
        lastY = ypos
        firstMouse = False

    xoffset = xpos - lastX
    yoffset = lastY - ypos
    lastX = xpos
    lastY = ypos

    sensitivity = 0.1 
    xoffset *= sensitivity
    yoffset *= sensitivity

    yaw += xoffset;
    pitch += yoffset;

    
    if pitch >= 90.0: pitch = 90.0
    if pitch <= -90.0: pitch = -90.0

    front = glm.vec3()
    front.x = math.cos(glm.radians(yaw)) * math.cos(glm.radians(pitch))
    front.y = math.sin(glm.radians(pitch))
    front.z = math.sin(glm.radians(yaw)) * math.cos(glm.radians(pitch))
    cameraFront = glm.normalize(front)

fov = 45.0

def scroll_event(window, xoffset, yoffset):
    global fov
    fov -= 2 * yoffset

glfw.set_input_mode(window, glfw.CURSOR, glfw.CURSOR_DISABLED);     # captura do mouse pela janela
glfw.set_key_callback(window,key_event)
glfw.set_cursor_pos_callback(window, mouse_event)
glfw.set_scroll_callback(window, scroll_event)

### Matrizes View e Projection

In [68]:
def view():
    global cameraPos, cameraFront, cameraUp
    mat_view = glm.lookAt(cameraPos, cameraPos + cameraFront, cameraUp);
    mat_view = np.array(mat_view)
    return mat_view

def projection():
    global altura, largura, fov
    # perspective parameters: fovy, aspect, near, far
    mat_projection = glm.perspective(glm.radians(fov), largura/altura, 0.1, 1000.0)
    mat_projection = np.array(mat_projection)    
    return mat_projection

### Loop principal
* Lógica para a rotação do avião
* Desenho dos objetos
* Controles: ASWD e mouse
* USE Alt-F4 PARA FECHAR A JANELA

In [69]:
glfw.show_window(window)
glfw.set_cursor_pos(window, lastX, lastY)

glEnable(GL_DEPTH_TEST)

# controle para a rotação do avião
plane_angle = 0


# intensidade da luz ambiente
loc_ambient_intensity = glGetUniformLocation(program, "ambientIntensity")

# intensidade da luz solar
loc_sun_intensity = glGetUniformLocation(program, "sunIntensity")

#[DEBUG] multiplicador da intensidade da luz 1 (interna) 
loc_light_one_intensity = glGetUniformLocation(program, "light1Intensity")

#[DEBUG] multiplicador da intensidade da luz 2 (externa)
loc_light_two_intensity = glGetUniformLocation(program, "light2Intensity")


loc_light_pos = glGetUniformLocation(program, "lightPos1") # recuperando localizacao da variavel lightPos na GPU
glUniform3f(loc_light_pos, 0, 0, 0)


loc_light_model = glGetUniformLocation(program, "lightPos2Model") # localizacao da matriz de transformacoes da luz externa
loc_light_pos = glGetUniformLocation(program, "lightPos2") # recuperando localizacao da variavel lightPos na GPU
glUniform3f(loc_light_pos, 0, 30, 0)
   

while not glfw.window_should_close(window):

    glfw.poll_events() 
    
    glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)
    
    glClearColor(1.0, 1.0, 1.0, 1.0)
    
    if polygonal_mode==True:
        glPolygonMode(GL_FRONT_AND_BACK,GL_LINE)
    if polygonal_mode==False:
        glPolygonMode(GL_FRONT_AND_BACK,GL_FILL)
        
        
    sun_intensity = max(math.cos(math.radians(sun_angle)), 0)
        
    # Lógica do avião. A rotação é constantemente decrementada
    # Uma matriz adicional de rotação é gerada e utilizada no passo
    # de desenho do avião
    plane_angle -= 0.05
    plane_extra_mat = Transform3D.rotacao(plane_angle, (0, 0, 0), 'y')
    
    
    # Lógica do sol. A rotação é constantemente decrementada
#     sun_angle -= 0.2
    sun_mat = Transform3D.rotacao(sun_angle, (0, 0, 0), 'z')

    # define matriz de rotacao da luz externa
    glUniformMatrix4fv(loc_light_model, 1, GL_TRUE, sun_mat)

    #Define a intensidade de cor do ambiente
    glUniform1f(loc_ambient_intensity, ambient_intensity)
    
    #Define a intensidade de luz do sol
    glUniform1f(loc_sun_intensity, sun_intensity)

    #[DEBUG] Define o multiplicador da luz 1
    glUniform1f(loc_light_one_intensity, light_one_intensity)

    #[DEBUG] Define o multiplicador da luz 2
    glUniform1f(loc_light_two_intensity, light_two_intensity)
    
    # Desenho dos objetos. Para o avião especificamente, uma matriz extra de rotação 
    # é computada para o desenho
    loc_model = glGetUniformLocation(program, "model")
    
    offset = 0
    for obj in all_objects:
        if(obj == plane):
            offset = obj.render(program, loc_model, offset, [plane_extra_mat])
        elif(obj == sun):
            offset = obj.render(program, loc_model, offset, [sun_mat])
        else:
            offset = obj.render(program, loc_model, offset)

    
    
    
    mat_view = view()
    loc_view = glGetUniformLocation(program, "view")
    glUniformMatrix4fv(loc_view, 1, GL_FALSE, mat_view)

    mat_projection = projection()
    loc_projection = glGetUniformLocation(program, "projection")
    glUniformMatrix4fv(loc_projection, 1, GL_FALSE, mat_projection)    
    
    # atualizando a posicao da camera/observador na GPU para calculo da reflexao especular
    loc_view_pos = glGetUniformLocation(program, "viewPos") # recuperando localizacao da variavel viewPos na GPU
    glUniform3f(loc_view_pos, cameraPos[0], cameraPos[1], cameraPos[2]) ### posicao da camera/observador (x,y,z)
    
    glfw.swap_buffers(window)

glfw.terminate()